<a href="https://colab.research.google.com/github/Naina-j04/Atri-Assignment/blob/main/TensorRT_Conversion_and_Inference_Benchmarking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import torch
import torchvision
from torch.utils.data import random_split
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.optim as optim
from PIL import Image
from collections import Counter
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


In [8]:
pip install onnx onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 110.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.3 MB/s eta 0:00:00


In [9]:
dummy_input = torch.randn(1, 3, 256, 256, device="cuda")
torch.onnx.export(model, dummy_input, "garbage.onnx", opset_version=11)


/tmp/ipython-input-3575621061.py:2: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(model, dummy_input, "garbage.onnx", opset_version=11)


In [1]:
!pip install tensorrt
!pip install onnxruntime-gpu


In [11]:
import onnxruntime as ort
import numpy as np

session = ort.InferenceSession(
    "garbage.onnx", providers=["CUDAExecutionProvider", "CPUExecutionProvider"]
)
input_name = session.get_inputs()[0].name
print("Input name:", dummy_input)

dummy_input = np.random.randn(1, 3, 256, 256).astype(np.float32)
outputs = session.run(None, {input_name: dummy_input})
print("Inference done, output shape:", outputs[0].shape)


Input name: tensor([[[[-1.4861, -0.5077, -1.1385,  ..., -1.7409, -0.7173,  0.8181],
          [-0.5270, -0.1506, -1.7448,  ..., -1.2186, -1.4222, -0.9324],
          [ 0.2534, -1.2258, -0.4874,  ...,  0.4911,  2.0538, -0.1398],
          ...,
          [ 0.4108,  0.5042, -0.6240,  ..., -0.3623, -2.5194, -0.1014],
          [-0.3738,  1.1263,  0.1817,  ...,  1.0988, -0.8465,  0.8414],
          [ 0.7424,  0.2869, -0.9102,  ...,  1.1698, -1.1110,  0.7320]],

         [[-0.2296, -0.2599,  0.8078,  ..., -0.8449,  0.1707, -2.1808],
          [ 2.3263,  1.3296, -0.0918,  ...,  0.3316, -0.3282, -1.1878],
          [ 0.6877,  0.8995,  0.2083,  ...,  0.6140, -0.2810, -0.4294],
          ...,
          [ 1.4194,  0.9867, -0.9599,  ..., -0.0082, -1.0392, -1.5934],
          [-0.2457,  0.7716,  0.8490,  ...,  0.8379,  0.3335, -0.3807],
          [-0.6915, -0.1015,  1.2668,  ...,  1.4560,  0.9314, -1.0864]],

         [[ 1.1179, -1.2905, -0.7018,  ...,  0.5777,  0.3957, -0.6929],
          [ 0.5541

In [10]:
model = models.resnet50()
path = '/content/drive/MyDrive/Garbage_classification/Data/model_resnet1.pth'
model.load_state_dict(torch.load(path))
model.eval()

dummy_input = torch.randn(1, 3, 256, 256)


#Pytorch CPU
model.cpu()
start = time.time()
with torch.no_grad():
    output_cpu = model(dummy_input)
cpu_time = time.time() - start
cpu_latency = cpu_time / batch_size
cpu_throughput = batch_size / cpu_time
print(f"PyTorch CPU: {cpu_latency:.4f}s/img, {cpu_throughput:.2f} img/s")

# Pytorch GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
dummy_input_gpu = dummy_input.to(device)
torch.cuda.synchronize()
start = time.time()
with torch.no_grad():
    output_gpu = model(dummy_input_gpu)
torch.cuda.synchronize()
gpu_time = time.time() - start
gpu_latency = gpu_time / batch_size
gpu_throughput = batch_size / gpu_time
print(f"PyTorch GPU: {gpu_latency:.4f}s/img, {gpu_throughput:.2f} img/s")


# TensorRT
session = ort.InferenceSession(
    "garbage.onnx", providers=["CUDAExecutionProvider", "CPUExecutionProvider"]
)
input_name = session.get_inputs()[0].name
dummy_input_np = dummy_input.numpy()

start = time.time()
outputs_trt = session.run(None, {input_name: dummy_input_np})
trt_time = time.time() - start
trt_latency = trt_time / batch_size
trt_throughput = batch_size / trt_time
print(f"TensorRT Model: {trt_latency:.4f}s/img, {trt_throughput:.2f} img/s")


PyTorch CPU: 0.0292s/img, 34.26 img/s
PyTorch GPU: 0.0009s/img, 1092.27 img/s
TensorRT Model: 0.0055s/img, 181.39 img/s
